# Pre-tutorial on Python for Digital Image Processing

This short Python tutorial introduces some general functions and practices needed for making the programming assignments for the Digital Image Processing course, like reading and displaying images. Please note that this notebook has been modified from [1].

### Setting up the environment 

First, we need to import a basic modules for reading and plotting images, and manipulating arrays:

In [ ]:
import matplotlib.pylab as plt # plotting package
from skimage import io         # utilities to read and write images in various formats
import numpy as np             # array manipulation package

(Note: If you need to familiarise yourself with arrays, please refer to the NumPy pre-tutorial.)

Run magic command `%matplotlib inline` in order to display the graphics inline:

In [ ]:
%matplotlib inline

Set the default size of figures in the notebook (unless otherwise specified):

In [ ]:
plt.rcParams['figure.figsize'] = (10, 5) # width and height

### Reading and plotting an image

Read an RGB image and display some of its properties:

In [ ]:
img = io.imread('mandril.png')

# variable type
print 'variable type:', type(img)

# data type
print 'data type:', img.dtype    

# print array shape/dimensions
print 'array shape:', img.shape  

# print number of array dimensions
print 'number of dimensions:', img.ndim  

# print number of elements in the array.
print 'number of elements:', img.size  

Plot the image:

In [ ]:
fig, ax = plt.subplots() # create figure
ax.imshow(img); # display image

The axes are useful in many cases but they can be also disabled:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img)
ax.axis('off'); # disable axes

In general, it is also good to give titles to images, especially if there are several subplots in the figure:

In [ ]:
fig, ax = plt.subplots()
ax.imshow(img)
ax.set_title('mandril.png') # set title
ax.axis('off');

It is also possible to override the default figure size if needed:

In [ ]:
fig, ax = plt.subplots(figsize=(10,6)) # create figure with custom size
ax.imshow(img)
ax.set_title('mandril.png')
ax.axis('off');

Subplots are often requested while making the assignments. Let's plot the different colour channels of the RGB image (sliced array), in the same figure:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3) # create figure with 1x3 subplots
ax[0].imshow(img[:,:,0]) # take the first channel which corresponds to red
ax[0].set_title('red channel')
ax[0].axis('off')
ax[1].imshow(img[:,:,1]) # take the second channel which corresponds to green
ax[1].set_title('green channel')
ax[1].axis('off')
ax[2].imshow(img[:,:,2]) # take the third channel which corresponds to blue
ax[2].set_title('blue channel') 
ax[2].axis('off');
fig.tight_layout() # automatically adjusts subplot params so that the subplots fit into the figure area

The default colormap lookup table (LUT) is `viridis`, thus the intensities of the single channels are shown in pseudocolor. The colormap of each subplot can be switched into grayscale: 

In [ ]:
fig, ax = plt.subplots(1, 3)
ax[0].imshow(img[:,:,0], cmap=plt.get_cmap('gray')) # set colormap to gray
ax[0].set_title('red channel')
ax[0].axis('off')
ax[1].imshow(img[:,:,1], cmap=plt.get_cmap('gray')) # set colormap to gray
ax[1].set_title('green channel')
ax[1].axis('off')
ax[2].imshow(img[:,:,2], cmap=plt.get_cmap('gray')) # set colormap to gray
ax[2].set_title('blue channel')
ax[2].axis('off');
fig.tight_layout()

Or one can simply change the default colormap into `gray`:

In [ ]:
plt.rcParams['image.cmap'] = 'gray' # set default colormap to gray

fig, ax = plt.subplots(1, 3)
ax[0].imshow(img[:,:,0])
ax[0].set_title('red channel')
ax[0].axis('off')
ax[1].imshow(img[:,:,1])
ax[1].set_title('green channel')
ax[1].axis('off')
ax[2].imshow(img[:,:,2])
ax[2].set_title('blue channel')
ax[2].axis('off');
fig.tight_layout() 

### Image data types and unexpected errors with arithmetic

Different image __[data types](http://scikit-image.org/docs/dev/user_guide/data_types.html)__ (`dtype`) are assumed to use the following ranges:
* `uint8` -> [0, 255]
* `uint16` ->[0, 65535]
* `uint32` -> [0, 232]
* `float` -> [-1, 1] or [0, 1]
* `int8` -> [-128, 127]
* `int16` -> [-32768, 32767]
* `int32` -> [-231, 231 - 1]

Therefore, image intensities should be always __[scaled](http://scikit-image.org/docs/dev/user_guide/data_types.html#rescaling-intensity-values)__ according to the data type in order to avoid distortion.

Many image processing functions e.g. in `skimage` are usually designed so that they accept any of the aforementioned data types but the returned image may be of different data type due to efficiency reasons. In `skimage`, there are utility functions for converting data types and rescaling the image intensities accordingly:
* `img_as_float` -> 64-bit floating point
* `img_as_ubyte` -> 8-bit uint
* `img_as_uint` -> 16-bit uint
* `img_as_int` -> 16-bit int

Please note that one should be very careful when using `astype` with images because it violates the aforementioned assumptions [2]:

In [ ]:
from skimage import img_as_float

image = np.arange(0, 255, 50, dtype=np.uint8)

print 'astype conversion:', image.astype(np.float) # These float values are out of range.

print 'conversion with scaling:', img_as_float(image)


Data types can be a source of "unexpected errors" when performing arithmetic operations, like addition and subtraction on images. Therefore, it is crucial to understand the different data types and the corresponding value ranges when one should have no problem recognising them.

NumPy uses modulo arithmetic on **overflow values** instead of clipping them.

In [ ]:
img = np.random.randint(0,255,[10,10], dtype='uint8') # 10x10 uint8 image with random values

# show original
data_plot = plt.imshow(img, vmin=0, vmax=255, interpolation='None')
plt.show()

# add 100 to each pixel value 
img100 = img + 100

# show the resulting image
data_plot = plt.imshow(img100, vmin=0, vmax=255, interpolation='None')
plt.show()

print 'data type after addition:', img100.dtype
print '\n'

print 'Original image:'
print img
print '\n'
print 'Note that result image values are calculated modulo 256, i.e. (img + 100) % 256:'
print img100
print '\n'

print 'img / 3 ... and floats get truncated toward zero:'
print img / 3

In order to get clipping behavior on overflow values, one can use a higher precision data types, for instance:

In [ ]:
img = np.random.randint(0,255,[10,10], dtype='uint16') # use higher precision, i.e. uint16

# show original
data_plot = plt.imshow(img, vmin=0, vmax=255, interpolation='None')
plt.show()

# add 100 to each pixel value
img100 = img + 100

# clip (saturate) resulting image and convert to uint8
img100uint8 = np.clip(img100, 0, 255).astype(np.uint8)

# add 100 to each pixel value and show resulting image
data_plot = plt.imshow(img100uint8, interpolation='None')
plt.show()

print 'data type after addition:', img100.dtype
print '\n'

print 'Original image:'
print img
print '\n'
print 'Note the values exceeding 255 in the resulting int16 image:'
print img100
print '\n'
print 'Clipped int16 image converted into uint8 (values over range saturated into 255):'
print img100uint8
print '\n'

Note that similar "issues" occur with negative values when subtracting images.

#### References

[1] https://github.com/karinsasaki/python-workshop-image-processing/blob/master/pre_tutorial/pre-tutorial.ipynb

[2] http://scikit-image.org/docs/dev/user_guide/data_types.html